In [ ]:
크롤러; 링크 수집기 + 색인을 위한 데이터 수집(스크래핑)
      ----
    * a[href]
* IFAME(네이버 블로그)[src]
      FORM
 IMG, VIDEO, AUDIO
 script, style, ...
    (불특정 다수)
%검색어% => 순위화, 관련성 측정 X

PageRank(Link 중 누가 중요한지 계산) => 모든 페이지가 동일하지 않다
                                => 우선 순위가 존재(가중치 표현)
                                   링크로 표현 가능

In [ ]:
*링크*
A -> B -> C -> A
A(B에 있는 내용이 좋거나 중요함)
지표가 될 수 있음

In [ ]:
URL Pool 존재
[thread Mananger]
while 더이상 URL이 없을 때까지,
    seed = URL.pop()
    thread => bot(seed)
    -> Link를 찾음
    -> 정규화할것임
    -> 새롭게 찾은 URL이면 Pool에 추가

In [5]:
from requests import request, get
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re

In [ ]:
# robots.txt Parser - 지난주 금요일


robots = dict() # 새로운 netloc(domain) 만나면, /robots.txt
                # 동일한                만나면, /robots.txt
                # robots = {'domain':['path', 'path', ...]}

# netloc, path(disallow에 있는지 검사할 대상)
def canFetch(host, path):
    # http/s로 시작해야지만 정상적인 URL, 검사
    if not re.search('^https?', host):
        return 'Scheme 오류'
    
    # URL에 제일 마지막에 /robots.txt 붙이기 위함
    # http://www.naver.com <= /robots.txt
    # http://www.naver.com/ <= robots.txt
    url = host+'/' if host[-1] !='/' else host
    url += 'robots.txt'
    
    # urlparse(http(s)://www.naver.com).netloc => k=www.naver.com
    k = urlparse(url).netloc
    # k not in robots{'www.naver.com':['path', 'path', ...]}
    # k(www.naver.com)이 dict에 없을 경우
    if k not in robots.keys():
        resp = get(url)
        if resp.status_code != 200:
            return resp.status_code

    if k not in robots.keys():
        # Disallow = '/'면 볼 필요 없음
        # {'google.com':[path, path, path, ...]}
        robots[k] = re.findall('^disallow:(.+)$', resp.text, re.IGNORECASE|re.MULTILINE)
        # 이 부분을 잘해야됨
    
    # k가 dict에 없는 경우, robots.txt 가져와서 dict[k] = ['path', 'path', ...]
    if path in robots[k]:
        return False
    else:
        # 명시적으로 거부하지 않았기에, opt-out(blacklist)에 의하여 True
        # path.split('/')
        # /search/adsf/asdfs # 없을 수 있음
        # /search 
        return True

In [ ]:
from time import sleep
# 앞으로 방문해야할 URL 목록
URLs = list() # URL Pool
# 기존에 방문한 URL 목록
Seens = list()

# 시작점이 되는 seed 주소 한개 추가
URLs.append('https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EC%B9%B4%EB%A6%AC%EB%82%98&oquery=zkflsk&tqi=i9MP%2FwqX5mNssc74FKCssssssPo-134918')
# Remote Disconnected 방지 or 상대방 사이트에서 bot감지 회피를 위하여 내 user-agent
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'}

# 더 이상 방문할 URL이 없을 때 까지
while URLs:
    # 방문할 url
    seed = URLs.pop(0) # Queue -> 너비 우선 탐색(BFS)
    # seed는 실제 방문을 하던, 아니면 오류나 중복이나 방문을 하지 않던, 방문한 목록에 추가
    Seens.append(seed)

    # robotsParser의 parameter를 위해
    # (scheme, netloc, path, params, qs, fragement)
    components = urlparse(seed)

    # canFetch('scheme://netloc', 'path')
    if canFetch('://'.join(components[:2]), components.path) == False:
        print('가져가면 안되요')

    # Request - Response
    resp = get(seed, headers=headers)

    # 400 or 500일 때
    if resp.status_code != 200:
        if resp.status_code == 500:
            URLs.append(seed)
        else:
            print('Response 없음')
            continue
    
    # text/*, application/*, image/*, ...
    # HyperLink in TEXT/HTML
    # HTML.Response.Headers에 TEXT/HTML인지 확인
    if not re.search(r'text/html', resp.headers['content-type']):
        continue
    
    # HTML => DOM
    dom = BeautifulSoup(resp.text, 'html.parser')
    #<a href="">, <button>, <form action="">, <iframe src="">, <audio>, <video>
    for link in dom.select('a[href], iframe[src]'):
        # 주소
        # 링크 추출
        href = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

        # 링크 어떻게 생겼는지
        # #top / javascript:func() / mailto: / tel: => 내용 X
        # URL 정규화, /다른페이지 => http(s)://netloc/다른페이지로 만들기 위함
        if not re.match(r'#|javascript', href):
            nurl = urljoin(seed, href)
            # 방문한 목록인 Seens에 없으면, 앞으로 방문할 URL
            if nurl not in Seens: # 방문한 페이지가 아니면
                URLs.append(nurl) 

    # 너무 많은 트래픽 발생시키지 않기 위함
    sleep(1) # <= 난수가 좋음

In [10]:
# robots.txt Parser - 지난주 금요일


robots = dict() # 새로운 netloc(domain) 만나면, /robots.txt
                # 동일한                만나면, /robots.txt
                # robots = {'domain':['path', 'path', ...]}
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'}


# netloc, path(disallow에 있는지 검사할 대상)
def canFetch(host, path):
    # http/s로 시작해야지만 정상적인 URL, 검사
    if not re.search('^https?', host):
        return 'Scheme 오류'
    
    # URL에 제일 마지막에 /robots.txt 붙이기 위함
    # http://www.naver.com <= /robots.txt
    # http://www.naver.com/ <= robots.txt
    url = host+'/' if host[-1] !='/' else host
    url += 'robots.txt'
    
    # urlparse(http(s)://www.naver.com).netloc => k=www.naver.com
    k = urlparse(url).netloc
    # k not in robots{'www.naver.com':['path', 'path', ...]}
    # k(www.naver.com)이 dict에 없을 경우
    if k not in robots.keys():
        resp = get(url, headers) # <= 수정
        # netloc/robots.txt 접근 x
        if resp.status_code != 200:
            print(resp.status_code)
            return True

    # if k not in robots.keys(): <= 수정
        # Disallow = '/'면 볼 필요 없음
        # {'google.com':[path, path, path, ...]}
        robots[k] = re.findall('^disallow:(.+)$', resp.text, re.IGNORECASE|re.MULTILINE)
        # 이 부분을 잘해야됨
    
    # k가 dict에 없는 경우, robots.txt 가져와서 dict[k] = ['path', 'path', ...]
    if path in robots[k]:
        return False
    else:
        # 명시적으로 거부하지 않았기에, opt-out(blacklist)에 의하여 True
        # path.split('/')
        # /search/adsf/asdfs # 없을 수 있음
        # /search 
        return True

In [18]:
from time import sleep
# 앞으로 방문해야할 URL 목록
URLs = list() # URL Pool
# 기존에 방문한 URL 목록
Seens = list()

# 시작점이 되는 seed 주소 한개 추가
URLs.append('https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EC%B9%B4%EB%A6%AC%EB%82%98&oquery=zkflsk&tqi=i9MP%2FwqX5mNssc74FKCssssssPo-134918')
# Remote Disconnected 방지 or 상대방 사이트에서 bot감지 회피를 위하여 내 user-agent
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'}

# 더 이상 방문할 URL이 없을 때 까지
while URLs:
    # 방문할 url
    seed = URLs.pop(0) # Queue -> 너비 우선 탐색(BFS)
    # seed = URLs.pop(-1) #Stack -> 깊이 우선 탐색(DFS)
    # seed는 실제 방문을 하던, 아니면 오류나 중복이나 방문을 하지 않던, 방문한 목록에 추가
    Seens.append(seed)

    # robotsParser의 parameter를 위해
    # (scheme, netloc, path, params, qs, fragement)
    components = urlparse(seed)

    # canFetch('scheme://netloc', 'path')
    if canFetch('://'.join(components[:2]), components.path) == False:
        print('가져가면 안되요')

    # Request - Response
    resp = get(seed, headers=headers)

    # 400 or 500일 때
    if resp.status_code != 200:
        if resp.status_code == 500:
            URLs.append(seed)
        else:
            print('Response 없음')
            continue
    
    # text/*, application/*, image/*, ...
    # HyperLink in TEXT/HTML
    # HTML.Response.Headers에 TEXT/HTML인지 확인
    if not re.search(r'text/html', resp.headers['content-type']):
        continue
    
    # HTML => DOM
    dom = BeautifulSoup(resp.text, 'html.parser')
    #<a href="">, <button>, <form action="">, <iframe src="">, <audio>, <video>
    for link in dom.select('a[href], iframe[src]'):
        # 주소
        # 링크 추출
        href = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

        # 링크 어떻게 생겼는지
        # #top / javascript:func() / mailto: / tel: => 내용 X
        # URL 정규화, /다른페이지 => http(s)://netloc/다른페이지로 만들기 위함
        if not re.match(r'#|javascript', href):
            nurl = urljoin(seed, href)
            # 방문한 목록인 Seens에 없으면, 앞으로 방문할 URL
            if nurl not in Seens and\
               nurl not in URLs: 
               # 방문한 페이지가 아니면
               # 방문할 목록인 URLs에 없으면, 앞으로 방문할 URL
                URLs.append(nurl) 

    # 너무 많은 트래픽 발생시키지 않기 위함
    # sleep(1) # <= 난수가 좋음

418
Response 없음
418
Response 없음


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [20]:
len(URLs), len(Seens)

(692, 15)

In [21]:
Seens

['https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EC%B9%B4%EB%A6%AC%EB%82%98&oquery=zkflsk&tqi=i9MP%2FwqX5mNssc74FKCssssssPo-134918',
 'https://www.naver.com',
 'https://search.naver.com/search.naver?ssc=tab.image.all&where=image&sm=tab_jum&query=%EC%B9%B4%EB%A6%AC%EB%82%98',
 'https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=%EC%B9%B4%EB%A6%AC%EB%82%98',
 'https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=%EC%B9%B4%EB%A6%AC%EB%82%98',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=%EC%B9%B4%EB%A6%AC%EB%82%98',
 'https://search.naver.com/search.naver?ssc=tab.kin.kqna&where=kin&sm=tab_jum&query=%EC%B9%B4%EB%A6%AC%EB%82%98',
 'https://search.naver.com/search.naver?ssc=tab.influencer.chl&where=influencer&sm=tab_jum&query=%EC%B9%B4%EB%A6%AC%EB%82%98',
 'https://search.naver.com/search.naver?ssc=tab.video.all&where=video&sm=tab_jum&query=%EC%B9%B4%EB%A6%AC%EB%82%98',


In [ ]:
웹 => 안끝남 => 전략(BFS, DFS, Focused Crawling)
Focused Crawling
1. Depth 제한
1       2       3 => Depth
A  ->  500 -> 10000
1      501 ->

2. Domain 제한
A -> 500.filter(naver.com)
     100 -> filter(naver.com)

3. 1 + 2

4. 영역(HTML Tag) 제한

In [35]:
# Focused Crawling
URLs = list()
Seens = list()

URLs.append({
    'url':'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EC%B9%B4%EB%A6%AC%EB%82%98&oquery=zkflsk&tqi=i9MP%2FwqX5mNssc74FKCssssssPo-134918',
    'depth': 1})
# Forcused Crawling
# 깊이 제한
limit = 3
# 도메인 제한
allow = [re.compile(re.escape('www.naver.com')),\
         re.compile(r'naver.com$')]

while URLs:
    seed = URLs.pop(-1)
    Seens.append(seed)

    # Focused Crawling 검사 부분
    # 깊이 제한
    if seed['depth'] > limit:
        continue
    # 도메인 제한
    if sum([m for m in map(lambda r:True if r.search(urlparse(seed['url']).netloc) else False, allow)]) == 0:
        # generator <= map(함수, iterator)
        # map(함수(iterable 객체):re.search() => if obj -> True else None -> True)
        # seed['url'] = 'blog.naver.com'
        # => [re.search(www.naver.com, blog.naver.com)=False,
        #     re.search(naver.com$, blog.naver.com)=True]
        # sum([False, True]) = 1
        # if sum == 0 이면 => WhiteList (해당 도메인만 방문)
        #         > 0 이면 => BlackList (해당 도메인 빼고 방문)
        continue
    components = urlparse(seed['url'])

    if canFetch('://'.join(components[:2]), components.path) == False:
        print('가져가면 안되요')

    resp = get(seed['url'], headers=headers)

    if resp.status_code != 200:
        if resp.status_code == 500:
            URLs.append(seed['url'])
        else:
            print('Response 없음')
            continue
    
    if not re.search(r'text/html', resp.headers['content-type']):
        continue
    
    dom = BeautifulSoup(resp.text, 'html.parser')
    for link in dom.select('a[href], iframe[src]'):
        href = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

        if not re.match(r'#|javascript', href):
            nurl = urljoin(seed['url'], href)
            if nurl not in [s['url'] for s in Seens] and\
               nurl not in [s['url'] for s in URLs]: 
               URLs.append({'url':nurl, 'depth':seed['depth']+1}) 

404
404
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
404
404
404
404
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음
Response 없음


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [38]:
len(URLs), len(Seens)

(208, 1367)

In [39]:
Seens

[{'url': 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EC%B9%B4%EB%A6%AC%EB%82%98&oquery=zkflsk&tqi=i9MP%2FwqX5mNssc74FKCssssssPo-134918',
  'depth': 1},
 {'url': 'https://www.naver.com/more.html', 'depth': 2},
 {'url': 'https://help.naver.com/', 'depth': 3},
 {'url': 'https://www.navercorp.com', 'depth': 4},
 {'url': 'https://help.naver.com/service/30016/contents/18033?lang=ko',
  'depth': 4},
 {'url': 'https://policy.naver.com/rules/youthpolicy.html', 'depth': 4},
 {'url': 'https://help.naver.com/index.help?lang=ko', 'depth': 4},
 {'url': 'https://policy.naver.com/policy/service_group.html', 'depth': 3},
 {'url': 'http://help.naver.com/', 'depth': 4},
 {'url': 'https://policy.naver.com/policy/emreject.html', 'depth': 4},
 {'url': 'https://www.navercorp.com/nhn/company/proposalGuide.nhn',
  'depth': 4},
 {'url': 'http://recruit.navercorp.com/', 'depth': 4},
 {'url': 'http://www.navercorp.com/', 'depth': 4},
 {'url': 'https://green.naver.com

In [58]:
URLs = list()
Seens = list()

URLs.append({
    'url': 'https://news.naver.com/',
    'depth': 1})

limit = 4
allow = [re.compile(re.escape('news.naver.com')),
         re.compile(r'news.naver.com$'),
         re.compile(r'pstatic.net$')]

while URLs:
    seed = URLs.pop(-1)
    Seens.append(seed)

    if seed['depth'] > limit:
        continue
    
    if sum([m for m in map(lambda r:True if r.search(urlparse(seed['url']).netloc) else False, allow)]) == 0:
        continue
    components = urlparse(seed['url'])

    if canFetch('://'.join(components[:2]), components.path) == False:
        print('가져가면 안되요')

    resp = get(seed['url'], headers=headers)

    if resp.status_code != 200:
        if resp.status_code == 500:
            URLs.append(seed['url'])
        else:
            print('Response 없음')
            continue

    print(seed['url'])

    # 링크+본문 Scraping 영역
    # image
    if re.search(r'image/(?:jpeg|jpg|gif|png|bmp)', resp.headers['content-type']):
        name = re.sub(r'[?:;/$]', '', seed['url'])
        ext = re.search(r'image/(jpeg|jpg|gif|png|bmp)', resp.headers['content-type']).group(1)
        with open(name+'.'+ext, 'wb') as fp:
            fp.write(resp.content)
    
    # text
    elif re.search(r'text/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        # HTML Tag 제한 => 
        # tagname:ul, class:Nlnb_menu_list의 자식 중 li
        # li의 자식 중 tagname:a, class:Nitem_link, 속성 href가 있는 링크
        # section100, 101, ...
        for link in dom.select('ul.Nlnb_menu_list li > a.Nitem_link[href]'):
            href = link.attrs['href']
            if not re.match(r'#|javascript', href):
                nurl = urljoin(seed['url'], href)
                if nurl not in [s['url'] for s in Seens] and\
                nurl not in [s['url'] for s in URLs]: 
                    URLs.append({'url':nurl, 'depth':seed['depth']+1}) 

        # 해당 selection에서의 뉴스링크
        for link in dom.select('div.sa_text > a[href]'):
            href = link.attrs['href']
            if not re.match(r'#|javascript', href):
                nurl = urljoin(seed['url'], href)
                if nurl not in [s['url'] for s in Seens] and\
                nurl not in [s['url'] for s in URLs]: 
                    URLs.append({'url':nurl, 'depth':seed['depth']+1}) 

        # Scraping
        if dom.select_one('#title_area, #dic_area'): # if None이면, 없다는뜻(뉴스 본문이 아님)
            # 본문 내 이미지 링크
            for link in dom.select('#dic_area img[data-src]'):
                href = link.attrs['data-src']
                if not re.match(r'#|javascript', href):
                    nurl = urljoin(seed['url'], href)
                    if nurl not in [s['url'] for s in Seens] and\
                    nurl not in [s['url'] for s in URLs]: 
                        URLs.append({'url':nurl, 'depth':seed['depth']+1}) 

            title = dom.select_one('#title_area')
            content = dom.select_one('#dic_area')
            g = re.search(r'(\d{3})/(\d{5,})$', seed['url'])
            with open(g.group(1)+'-'+g.group(2)+'.txt', 'w', encoding='utf8') as fp:
                fp.write(title.text+'\n\n')
                fp.write(content.text)

https://news.naver.com/
https://news.naver.com/ombudsman/errorArticleList
https://news.naver.com/factcheck/main
https://news.naver.com/main/tv/index.naver?mid=tvh
https://news.naver.com/opinion/home
https://news.naver.com/newspaper/home?viewType=pc
https://news.naver.com/main/ranking/popularDay.naver
https://news.naver.com/section/104
https://n.news.naver.com/mnews/article/008/0005166418
404
https://imgnews.pstatic.net/image/008/2025/03/17/0005166418_002_20250317083110534.jpg?type=w860
404
https://imgnews.pstatic.net/image/008/2025/03/17/0005166418_001_20250317083110497.jpg?type=w860
https://n.news.naver.com/mnews/article/081/0003525603
404
https://imgnews.pstatic.net/image/081/2025/03/17/0003525603_003_20250317105013569.png?type=w860
404
https://imgnews.pstatic.net/image/081/2025/03/17/0003525603_002_20250317105013244.gif?type=w860
404
https://imgnews.pstatic.net/image/081/2025/03/17/0003525603_001_20250317105013209.jpg?type=w860
https://n.news.naver.com/mnews/article/005/0001763476
4

In [59]:
len(URLs), len(Seens)

(0, 661)

In [60]:
Seens

[{'url': 'https://news.naver.com/', 'depth': 1},
 {'url': 'https://news.naver.com/ombudsman/errorArticleList', 'depth': 2},
 {'url': 'https://media.naver.com/algorithm', 'depth': 2},
 {'url': 'https://news.naver.com/factcheck/main', 'depth': 2},
 {'url': 'https://news.naver.com/main/tv/index.naver?mid=tvh', 'depth': 2},
 {'url': 'https://news.naver.com/opinion/home', 'depth': 2},
 {'url': 'https://news.naver.com/newspaper/home?viewType=pc', 'depth': 2},
 {'url': 'https://news.naver.com/main/ranking/popularDay.naver', 'depth': 2},
 {'url': 'https://news.naver.com/section/104', 'depth': 2},
 {'url': 'https://n.news.naver.com/mnews/article/008/0005166418', 'depth': 3},
 {'url': 'https://imgnews.pstatic.net/image/008/2025/03/17/0005166418_002_20250317083110534.jpg?type=w860',
  'depth': 4},
 {'url': 'https://imgnews.pstatic.net/image/008/2025/03/17/0005166418_001_20250317083110497.jpg?type=w860',
  'depth': 4},
 {'url': 'https://n.news.naver.com/mnews/article/081/0003525603', 'depth': 3},


# 크롤러는 링크 따라간다